In [ ]:
import numpy as np
import pandas as pd

In [ ]:
movies=pd.read_csv('tmdb_5000_movies.csv')
credits=pd.read_csv('tmdb_5000_credits.csv')

In [ ]:
movies.head()


In [ ]:
credits.head()

In [ ]:
movies.info()

In [ ]:
movies =movies.merge(credits, on='title')

In [ ]:
movies.info()

In [ ]:
movies = movies[['id', 'title', 'overview', 'genres', 'keywords', 'cast', 'crew']]


In [ ]:
movies.head()

In [ ]:

movies.isnull().sum()

In [ ]:
movies.dropna(inplace=True)

In [ ]:
movies.isnull().sum()

In [ ]:
movies.iloc[0].genres

In [ ]:
#'[{"id": 28, "name": "Action"}, {"id": 12, "name": "Adventure"}, {"id": 14, "name": "Fantasy"}, {"id": 878, "name": "Science Fiction"}]'
#['Action', 'adventure','Fantasy',''Scifi']

In [ ]:
def convert(obj):
    L = []
    for i in ast.literal_eval(obj):
        L.append(i['name'])  
    return L  

In [ ]:
import ast
ast.literal_eval('[{"id": 28, "name": "Action"}, {"id": 12, "name": "Adventure"}, {"id": 14, "name": "Fantasy"}, {"id": 878, "name": "Science Fiction"}]'
)

In [ ]:
movies['genres']= movies['genres'].apply(convert)

In [ ]:
movies.head()

In [ ]:
movies['cast'][0]

In [ ]:
movies['keywords']= movies['keywords'].apply(convert)

In [ ]:
movies.head()

In [ ]:
def convert3(obj):
    L =[]
    counter = 0
    for i in ast.literal_eval(obj): 
        if counter != 3:
             L.append(i['name'])
             counter +=1
        else:
             break
    
    return L

In [ ]:
movies['cast'] = movies['cast'].apply(convert3)


In [ ]:
movies['cast'].head()

In [ ]:
movies['crew'][0]

In [ ]:
def fetch_director(obj):
    L =[]
    for i in ast.literal_eval(obj):
        if i['job'] == 'Director':  
            L.append(i['name'])
            break
    return L

In [ ]:
movies['crew'] = movies['crew'].apply(fetch_director) 

In [ ]:
movies.head()

In [ ]:
 movies['overview'] = movies['overview'].apply(lambda x:x.split())

In [ ]:
movies['genres'].apply(lambda x: [i.replace(" ", "") for i in x])

In [ ]:
movies.head()

In [ ]:
movies['genres'] = movies['genres'].apply(lambda x: [i.replace(" ", "") for i in x] )
movies['keywords'] = movies['keywords'].apply(lambda x: [i.replace(" ", "") for i in x])
movies['cast'] = movies['cast'].apply(lambda x: [i.replace(" ", "") for i in x])
movies['crew'] = movies['crew'].apply(lambda x: [i.replace(" ", "") for i in x])


movies.head()

In [ ]:
movies['tags'] = movies['overview']+ movies['genres']+ movies['keywords'] + movies['cast']+ movies['crew']

In [ ]:
movies.head()

In [ ]:
new_df = movies[['id', 'title', 'tags']]

In [ ]:
new_df


In [ ]:
new_df['tags'] = new_df['tags'].apply(lambda X: " ".join(X))


In [ ]:
new_df.head()

In [ ]:
new_df['tags'][0]

In [ ]:
new_df['tags'] = new_df['tags'].apply(lambda X:X .lower())

In [ ]:
new_df.head()

In [ ]:
new_df['tags'][0]

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer(max_features=5000, stop_words='english')


In [ ]:
vectors = cv.fit_transform(new_df['tags']).toarray()

In [ ]:
vectors

In [ ]:
vectors[0]

In [ ]:
cv.get_feature_names_out()

In [ ]:
import nltk

In [ ]:
new_df.head()

In [ ]:
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

In [ ]:
def stem(text):
    y =[]
    
    for i in text.split():
        y.append(ps.stem(i))
    return  " ".join(y)

In [ ]:
new_df['tags'].apply(stem)

In [ ]:
['loved','loving','love']
['love','love','love']

In [ ]:
stem('in the 22nd century, a paraplegic marine is dispatched to the moon pandora on a unique mission, but becomes torn between following orders and protecting an alien civilization. action adventure fantasy sciencefiction cultureclash future spacewar spacecolony society spacetravel futuristic romance space alien tribe alienplanet cgi marine soldier battle loveaffair antiwar powerrelations mindandsoul 3d samworthington zoesaldana sigourneyweaver jamescameron')


In [ ]:
from sklearn.metrics.pairwise import cosine_similarity


In [ ]:
similarity = cosine_similarity(vectors)


In [ ]:
movies_list = sorted(list(enumerate(similarity[0])),reverse=True, key = lambda x:x[1:6])

In [ ]:
sorted(similarity[0], reverse=True)


In [ ]:
def recommend(movie):
    
    movie_index = new_df[new_df['title'] == movie].index[0] 

   
    distances = similarity[movie_index]

   
    movie_list = sorted(enumerate(distances), key=lambda x: x[1], reverse=True)[1:6]  

    for i in movie_list:
        print(new_df.iloc[i[0]]['title'])  

In [ ]:
recommend('Batman Begins')

In [ ]:
def recommend(movie):
    
    movie_index = new_df[new_df['title'] == movie].index[0]  
    
    
    distances = similarity[movie_index]  
    sorted_distances = sorted(list(enumerate(distances)), reverse=True, key=lambda x: x[1])
    
    
    for i in sorted_distances[1:6]:  
        print(new_df.iloc[i[0]]['title'])

In [ ]:
new_df.head()


In [ ]:
recommend('Avatar')

In [ ]:
import pickle


In [ ]:

with open('movies.pkl', 'wb') as f:
    pickle.dump(new_df, f)


with open('movies_dict.pkl', 'wb') as f:
    pickle.dump(new_df.to_dict(), f)


In [ ]:
with open('similarity.pkl', 'wb') as file:
    pickle.dump(similarity, file)
